In [ ]:
!pip install accelerate==0.29.3
!pip install bitsandbytes==0.43.1
!pip install transformers==4.40
!pip install peft==0.10.0

In [ ]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging
)
from transformers import AutoConfig,AutoModel
import torch
from peft import PeftModel


In [ ]:
import huggingface_hub
huggingface_hub.login()

# 1. Load base and new model

In [ ]:
## set base model
base_model = "beomi/Llama-3-Open-Ko-8B"
# base_model = "meta-llama/Meta-Llama-3-8B-Instruct"
## set your fined tuned model path (ex: google drive folder)
# new_model = "/content/gdrive/MyDrive/Colab Notebooks/llama/02. Fine Tuning/Llama-hkcode-Ko-8b"
#new_model = "/content/gdrive/MyDrive/Colab Notebooks/llama/02. Fine Tuning/llama3_meta_hkcode_0602"
new_model = "finetuning/llama3_rscode_1013_100"
### Load basemodel
baseModel = AutoModelForCausalLM.from_pretrained(
    base_model,
    low_cpu_mem_usage=True,
    return_dict=True,torch_dtype=torch.float16,
    device_map= {"": 0}
)

### Load basemodel's tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
!ls finetuning/llama3_rscode_1013_100

In [ ]:
# merge model
model = PeftModel.from_pretrained(baseModel, new_model)

### 어뎁터 연결 모델 정상작동 확인

In [ ]:
def generate_response(prompt, model):
    encoded_input = tokenizer(prompt,  return_tensors="pt", add_special_tokens=True)
    model_inputs = encoded_input.to('cuda')

    generated_ids = model.generate(**model_inputs, max_new_tokens=512, do_sample=True, pad_token_id=tokenizer.eos_token_id)
    # generated_ids = model.generate(**model_inputs, max_new_tokens=200, do_sample=True, pad_token_id=128009)

    decoded_output = tokenizer.batch_decode(generated_ids)

    return decoded_output[0].replace(prompt, "")
key="BTS는 누구인가요?"
prompt=f"""you are a assistant please answer in korean lanauage

### Instruction:
{key}

### Response:"""
generate_response(prompt, model)

In [ ]:
# logging.set_verbosity(logging.CRITICAL)

# prompt = "HKCODE 유튜브 채널은 누가 운영하나요?"
# pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=100)
# result = pipe(f"<s>[INST] {prompt} [/INST]")
# print(result[0]['generated_text'])
# # trainer.save_model(new_model)

In [ ]:
mergedModel = model.merge_and_unload()

# set your HF repository
hfAddr = "rssaem/llama3_ko_8b_rs1"

# save model and tokenizer to HF hub
mergedModel.push_to_hub(hfAddr)
tokenizer.push_to_hub(hfAddr)
# tokenizer.push_to_hub(hfAddr, use_temp_dir=False)